# Global indicator project - Phoenix, Arizona

## local neighborhood network accessibility analysis - amenities pois
This notebook uses sausage buffer intersection approach to calculate the accessibility score of sample point local neighborhood. This process use hard threshold distance to count whether any pois (in this case, it is shops with supermarket and convenience stores) located within the local walkable neighborhoods.

### Process:
1. Download or load points of interest (POIs) from OSM
2. Get sample point local neighborhood (sausage buffer)
3. Local neighbothood with at least one amenity pois intersecting
    - the output results are geoseries of local neighborhood buffer polygon
4. Connect the result to sample point dataframe
    - accessibility score is calculated as 1=at least one amenity pois is within the local walkable neighborhood; 0=none of the pois is within the local walkable neighborhood.




In [2]:
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
import numpy as np
import requests
import pandas as pd
import geopandas as gpd
import os
import time 
from shapely.geometry import shape,Point, LineString, Polygon


import pandana
from pandana.loaders import osm

ox.config(use_cache=True, log_console=True)
pandana.__path__ #pandana runs on python 3.6 kernel

['/Users/NGAU/miniconda3/envs/ind_global/lib/python3.6/site-packages/pandana']

## Set up configuration 

In [4]:
place = 'phoenix' 

region = 'Arizona, USA' # study region name

studyregion = 'Phoenix, Arizona, USA'

suffix = '_201905'

# configure search at a max distance of 1 km for up to the 10 nearest points-of-interest
shop = ['supermarket', 'convenience']

# configure filenames to save/load POI and network datasets
OSM_folder = '../data/OSM'

G_filename = '{studyregion}_walk{suffix}.graphml'.format(studyregion = studyregion, suffix = suffix)
G_proj_filename='{studyregion}_proj_walk{suffix}.graphml'.format(studyregion = place, suffix = suffix)
poi_filename = '{}_pois_{}.csv'.format(place, '_'.join(shop))

G_filepath = OSM_folder + "/" + G_filename
poi_filepath = OSM_folder + "/" + poi_filename
sample_points_filepath = '../data/OSM/phoenix_sample_points_stats_201905/phoenix_sample_points_stats_201905.shp'

In [5]:
# get bounding box from study region boundary shapefile
shape_filepath = '../data/OSM/Phoenix, Arizona, USA_buffered_201905/Phoenix, Arizona, USA_buffered_201905.shp'

gdf_shape = gpd.GeoDataFrame.from_file(shape_filepath)
bbox = [gdf_shape['bbox_south'].astype(float)[0], gdf_shape['bbox_west'].astype(float)[0], gdf_shape['bbox_north'].astype(float)[0], gdf_shape['bbox_east'].astype(float)[0]] #lat-long bounding box for Phx
bbox

[33.2903739, -112.3240289, 33.9183794, -111.9255201]

## Download points of interest (POIs) from OSM
What amenities are considered for daily living pois? - [OSMtag](https://taginfo.openstreetmap.org/keys/amenity): shop=supermarket, convenience

In [6]:
def get_osm_pois_gdf(poi_filepath=poi_filepath, shop=shop, bbox=bbox):
    if os.path.isfile(poi_filepath):
        # if a points-of-interest file already exists, just load the dataset from that
        pois = pd.read_csv(poi_filepath)
        method = 'loaded from CSV'
    else:   
        # otherwise, query the OSM API for the specified amenities within the bounding box 
        osm_tags = '"shop"~"{}"'.format('|'.join(shop))
        pois = osm.node_query(bbox[0], bbox[1], bbox[2], bbox[3], tags=osm_tags)

        # drop any that aren't just 'shop' then save to CSV
        pois = pois[pois['shop'].isin(shop)]
        pois.to_csv(poi_filepath, index=False, encoding='utf-8')
        method = 'downloaded from OSM'
    pois_df = pois[['shop', 'name', 'lat', 'lon']]
    pois_df['geometry'] = pois_df.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
    pois_gdf = gpd.GeoDataFrame(pois_df)
    return pois_gdf
    

In [7]:
pois_gdf = get_osm_pois_gdf(poi_filepath=poi_filepath, shop=shop, bbox=bbox)
pois_gdf.head()

/Users/NGAU/miniconda3/envs/ind_global/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,shop,name,lat,lon,geometry
0,supermarket,Safeway,33.488551,-112.081794,POINT (-112.0817935 33.4885505)
1,convenience,7-Eleven,33.480013,-112.029747,POINT (-112.0297471 33.4800128)
2,supermarket,Fry's Marketplace,33.321899,-111.930303,POINT (-111.9303033 33.3218992)
3,supermarket,Fry's Food & Drug,33.379098,-111.976857,POINT (-111.9768574 33.3790977)
4,convenience,NaN,33.348719,-111.953919,POINT (-111.9539185 33.3487189)


In [8]:
#define pois points projection
pois_gdf.crs = {'init' :'epsg:4326'}
pois_gdf_proj = ox.project_gdf(pois_gdf)

## Get sample point local neighborhood (sausage buffer)

In [9]:
# load sample point dataframe
sample_points = gpd.GeoDataFrame.from_file(sample_points_filepath)
sample_points.shape

(100, 27)

In [10]:
# load study region projected graph
G_proj = ox.load_graphml(G_proj_filename, folder=OSM_folder)

In [11]:
# create list of sample points to iterate over
point_locations = []

for point in sample_points.geometry: 
    point = (point.x, point.y)
    point_locations = point_locations + [point]

In [14]:
# create sausage buffer local neighborhood
def create_sausage_buffer_gdf(G_proj, orig_point, buffer=50, length = 1600, intersection_tolerance = 15):
    # locate closest node on network to 
    orig_node = ox.get_nearest_node(G_proj, orig_point, return_dist=True)
    subgraph_proj = nx.ego_graph(G_proj, orig_node[0], radius=length, distance='length')
    subgraph_gdf = ox.graph_to_gdfs(subgraph_proj, nodes=False, edges=True, fill_edge_geometry=True)
    # create buffer polygon geometry to dataframe
    subgraph_gdf['geometry'] = subgraph_gdf.geometry.buffer(buffer) 
    #link original node id reference
    subgraph_gdf['node_id'] = orig_node[0]
    return(subgraph_gdf) #output is smaple point subgraph with buffer polygon geometry and original node id reference

In [15]:
# iterate over a list of sample points to create sausage buffer as local walkable neighborhood
start = time.time()
task = 'Buffer network for {} sample points'.format(len(point_locations))
sausagebuffers = []
for point in point_locations:
    sausagebuffers.append(create_sausage_buffer_gdf(G_proj, point))
print('Completed task "{}" in {:,.2f} seconds'.format(task,time.time() - start)) 

Completed task "Buffer network for 100 sample points" in 95.92 seconds


## POIs point and sausage buffer intersection

In [25]:
# make sure pois geometry is in the same projection as the sausage buffer geometry
pois_gdf_proj = pois_gdf_proj.to_crs(sausagebuffers[0].crs)
pois_gdf_proj.crs == sausagebuffers[0].crs

True

In [26]:
# create an empty list
buffer_node_id = pd.DataFrame()
# loop through sausage buffer intersection with pois
for x in range(0, len(sausagebuffers)):
    buffer_node_id1 = pd.DataFrame()
    #Returns a Series of dtype('bool') with value True for each polygon geometry that intersects other.
    pois_intersect = sausagebuffers[x].geometry.intersects(pois_gdf_proj['geometry'].unary_union) 
    # return a dataframe with pois within local walkable neighborhood (pois intersecting buffer), 
    # retain orignal node id for reference in sample point neighborhood
    buffer_node_id1['pois_walkable_node_id'] = [sausagebuffers[x][pois_intersect]['node_id'].max()]
    buffer_node_id = buffer_node_id.append(buffer_node_id1, ignore_index=True).dropna().astype(int)

## Connect the result in sample point dataframe

In [28]:
# load previously saved sample point dataframe
sample_points.columns

Index(['index', 'Series', 'access', 'area', 'bridge', 'highway', 'junction',
       'key', 'lanes', 'length', 'maxspeed', 'name', 'oneway', 'osmid', 'ref',
       'service', 'tunnel', 'u', 'v', 'width', 'points', 'area_km',
       'clean_inte', 'clean_in_1', 'origin_nod', 'origin_n_1', 'geometry'],
      dtype='object')

In [66]:
buffer_node_id = buffer_node_id.drop_duplicates('pois_walkable_node_id')

In [71]:
# merge sample point datafram and buffer with pois intersection based on node id to get the accessibility distance
sample_points.origin_n_1 = sample_points.origin_n_1.astype(int)
sample_points_stats = pd.merge(left=sample_points, right=buffer_node_id, how='left', left_on='origin_n_1', right_on='pois_walkable_node_id', validate="many_to_one")

In [73]:
sample_points_stats = sample_points_stats.rename(columns={'pois_walkable_node_id':'nearest_shop_walkable'})
#fill none value as 0 indicating that the nearest shop is not within walkable network buffer of the nodes/sample point
#fill values more than 0 as 1 indicating that the nearest shop is within walkable network buffer of the nodes/sample point
sample_points_stats['nearest_shop_walkable'] = sample_points_stats['nearest_shop_walkable'].fillna(0) 
sample_points_stats.loc[sample_points_stats['nearest_shop_walkable'] > 0, 'nearest_shop_walkable'] = 1
sample_points_stats.head()

,index,Series,access,area,bridge,highway,junction,key,lanes,length,...,v,width,points,area_km,clean_inte,clean_in_1,origin_nod,origin_n_1,geometry,nearest_shop_walkable
0,0,POINT (399320.4111184602 3704054.876976978),None,None,None,residential,None,0,2,50.876,...,1515772851,None,[<shapely.geometry.point.Point object at 0x1a2...,3.776544590718806,406,107.5056815157912,18094.224675152538,41797604,POINT (399320.4111184602 3704054.876976978),1.0
1,0,POINT (399294.9269077833 3704055.126239192),None,None,None,residential,None,0,2,50.876,...,1515772851,None,[<shapely.geometry.point.Point object at 0x1a2...,4.653264340395821,544,116.90717745764809,35275.866266879544,1604511752,POINT (399294.9269077833 3704055.126239192),1.0
2,0,POINT (399269.4426971063 3704055.375501406),None,None,None,residential,None,0,2,50.876,...,1515772851,None,[<shapely.geometry.point.Point object at 0x1a2...,3.349992218481829,319,95.22410178748609,24521.116301322858,5634577087,POINT (399269.4426971063 3704055.375501406),1.0
3,1,POINT (399320.4111184602 3704054.876976978),None,None,None,residential,None,0,2,17.429,...,1515772854,None,[<shapely.geometry.point.Point object at 0x1a2...,3.776544590718806,406,107.5056815157912,18094.224675152538,41797604,POINT (399320.4111184602 3704054.876976978),1.0
4,1,POINT (399337.8713994289 3704054.705951796),None,None,None,residential,None,0,2,17.429,...,1515772854,None,[<shapely.geometry.point.Point object at 0x1a2...,3.556235791929595,349,98.13747468376792,4233.820518541409,2926772436,POINT (399337.8713994289 3704054.705951796),1.0


In [74]:
sample_points_stats.shape

(100, 28)